<img src='imgs/logstash.png' width='750'/>

# c* + elk

In [23]:
elastic_version = '6.6.2'

# c* jdbc driver

In [24]:
!echo 'verify location:'
!ls -l   logstash_vol/cassandra-jdbc-driver-1.3.jar

!echo
!echo 'verify class name:'
!jar tvf logstash_vol/cassandra-jdbc-driver-1.3.jar | grep CassandraJdbcDriver

verify location:
-rw-r--r--  1 Bhill  staff  7645422 Apr 24 10:06 logstash_vol/cassandra-jdbc-driver-1.3.jar

verify class name:
  6211 Mon Feb 11 15:37:56 EST 2019 com/dbschema/CassandraJdbcDriver.class


# logstash config

__the lack of cron schedule indicates this will run and quit__

In [3]:
!cat logstash_vol/logstash.cassandra.conf

input {
  jdbc {
    jdbc_driver_library    => "/config-dir/cassandra-jdbc-driver-1.3.jar"
    jdbc_driver_class      => "com.dbschema.CassandraJdbcDriver"
    jdbc_connection_string => "jdbc:cassandra://cassandra:9042/elk_c"
    jdbc_user              => "root"
    statement              => "select id, city, name, phone, salary from employee"
  }
}
output {  
    elasticsearch {  
	hosts => ["elasticsearch:9200"]  
	index => "employees"
    }  
}  


# bulk load & quit

In [4]:
!docker run -h logstash                            \
            --name logstash                        \
            --link cassandra:cassandra             \
            --link elasticsearch:elasticsearch     \
            -it --rm                               \
            -v ${PWD}/logstash_vol:/config-dir     \
            logstash:6.6.2                         \
            -f /config-dir/logstash.cassandra.conf

Sending Logstash logs to /usr/share/logstash/logs which is now configured via log4j2.properties
[2019-04-24T17:54:11,689][INFO ][logstash.setting.writabledirectory] Creating directory {:setting=>"path.queue", :path=>"/usr/share/logstash/data/queue"}
[2019-04-24T17:54:11,702][INFO ][logstash.setting.writabledirectory] Creating directory {:setting=>"path.dead_letter_queue", :path=>"/usr/share/logstash/data/dead_letter_queue"}
[2019-04-24T17:54:12,371][WARN ][logstash.config.source.multilocal] Ignoring the 'pipelines.yml' file because modules or command line options are specified
[2019-04-24T17:54:12,392][INFO ][logstash.runner          ] Starting Logstash {"logstash.version"=>"6.6.2"}
[2019-04-24T17:54:12,444][INFO ][logstash.agent           ] No persistent UUID file found. Generating new UUID {:uuid=>"16a5e62f-45a1-4672-a441-9e2bf459cf63", :path=>"/usr/share/logstash/data/uuid"}
[2019-04-24T17:54:13,440][WARN ][logstash.monitoringextension.pipelineregisterhook] xpack.monitoring.enabled 

[2019-04-24T17:54:23,772][INFO ][logstash.inputs.jdbc     ] (0.027487s) select id, city, name, phone, salary from employee
[2019-04-24T17:54:23,873][WARN ][logstash.inputs.jdbc     ] Exception when executing JDBC query {:exception=>#<Sequel::DatabaseError: Java::ComDatastaxDriverCoreExceptions::CodecNotFoundException: Codec not found for requested operation: [varint <-> java.lang.Integer]>}
[2019-04-24T17:54:24,653][INFO ][logstash.pipeline        ] Pipeline has terminated {:pipeline_id=>"main", :thread=>"#<Thread:0x7b823337 run>"}
[2019-04-24T17:54:26,780][INFO ][logstash.pipeline        ] Pipeline has terminated {:pipeline_id=>".monitoring-logstash", :thread=>"#<Thread:0x2eee3e11 run>"}


# kibana validation

<img src='imgs/kibana_timelion.png' width=450 />